# Proceeds Comparison Analysis

This notebook reads all CSV files from the output folder and compares `eur_proceeds_d8` with `expected_proceeds_d8`, as well as the actual and expected proceeds share on the `channel_group` and `user_type` level.

In [2]:
# Import necessary libraries
import pandas as pd
import os
from glob import glob

# Define the path to the output folder
output_folder = 'output'

# Read all CSV files in the output folder
csv_files = glob(os.path.join(output_folder, '*.csv'))

# Concatenate all CSV files into a single DataFrame
dataframes = [pd.read_csv(file) for file in csv_files]
df = pd.concat(dataframes, ignore_index=True)

print(f"Loaded {len(df):,} records from {len(csv_files)} CSV files.")

Loaded 141,881 records from 4 CSV files.


In [6]:
# Import necessary libraries
from snowflake.snowpark import Session
import config
import pandas as pd

# Connect to Snowflake
def get_snowflake_session():
    connection_parameters = {
        "account": config.SNOWFLAKE_ACCOUNT,
        "user": config.SNOWFLAKE_USER,
        "role": config.SNOWFLAKE_ROLE,
        "warehouse": config.SNOWFLAKE_WAREHOUSE,
        "database": config.SNOWFLAKE_DATABASE,
        "schema": config.SNOWFLAKE_SCHEMA,
        "authenticator": config.SNOWFLAKE_AUTHENTICATOR
    }
    session = Session.builder.configs(connection_parameters).create()
    print(f"Connected to Snowflake as {config.SNOWFLAKE_USER}")
    return session

# Initialize Snowflake session
session = get_snowflake_session()

# Define source table name
source_table = "BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250314"

# Read data from Snowflake
print(f"Reading data from {source_table}...")
query = f"SELECT * FROM {source_table}"
df = session.sql(query).to_pandas()
print(f"Read {len(df):,} rows from {source_table}")

# Close Snowflake session
session.close()
print("Snowflake session closed")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://blinkist-useast_1_virginia.snowflakecomputing.com/console/login?login_name=meri-kris.jaama%40go1.com&browser_mode_redirect_port=54503&proof_key=xZv1SutfZQUn10cw2lhZkb7uYHgco39spVQXRAOveAc%3D to authenticate...
Connected to Snowflake as meri-kris.jaama@go1.com
Reading data from BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250314...
Read 215,999 rows from BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250314
Snowflake session closed


## Aggregate and Compare Proceeds

In [9]:
# Group by channel_group and user_type
grouped_df = df.groupby(['channel_group', 'user_type']).agg({
    'eur_proceeds_d8': 'sum',
    'expected_proceeds_d8': 'sum'
}).reset_index()

# Calculate total proceeds for each channel_group
total_actual_proceeds_by_channel = grouped_df.groupby('channel_group')['eur_proceeds_d8'].transform('sum')
total_expected_proceeds_by_channel = grouped_df.groupby('channel_group')['expected_proceeds_d8'].transform('sum')

# Calculate actual and expected proceeds share within each channel_group
grouped_df['actual_proceeds_share'] = (grouped_df['eur_proceeds_d8'] / total_actual_proceeds_by_channel) * 100
grouped_df['expected_proceeds_share'] = (grouped_df['expected_proceeds_d8'] / total_expected_proceeds_by_channel) * 100

# Calculate the error between expected and actual proceeds
grouped_df['proceeds_error'] = grouped_df['expected_proceeds_d8'] - grouped_df['eur_proceeds_d8']
grouped_df['error_ratio'] = grouped_df['expected_proceeds_d8'] / grouped_df['eur_proceeds_d8']
# Display the results
print("Comparison of Actual and Expected Proceeds:")
display(grouped_df)

Comparison of Actual and Expected Proceeds:


,channel_group,user_type,eur_proceeds_d8,expected_proceeds_d8,actual_proceeds_share,expected_proceeds_share,proceeds_error,error_ratio
0,affiliate_and_partnerships,day0_payer,15936.178232,15936.178232,92.151470,94.896971,0.000000,1.000000
1,affiliate_and_partnerships,other,886.336087,548.128166,5.125267,3.264001,-338.207921,0.618420
2,affiliate_and_partnerships,trial,470.946343,308.830526,2.723263,1.839028,-162.115817,0.655766
3,brand_search,day0_payer,84520.334719,84520.334719,78.481698,82.743034,0.000000,1.000000
4,brand_search,other,11284.152912,6362.017604,10.477946,6.228237,-4922.135309,0.563801
5,brand_search,trial,11889.836502,11265.623888,11.040356,11.028729,-624.212614,0.947500
6,display_and_programmatic,day0_payer,164256.559902,164256.559902,69.692375,72.545233,0.000000,1.000000
7,display_and_programmatic,other,27839.312113,12722.960539,11.811935,5.619198,-15116.351574,0.457014
8,display_and_programmatic,trial,43592.121078,49439.987628,18.495690,21.835569,5847.866550,1.134150
9,generic_search,day0_payer,21083.422776,21083.422776,77.991034,84.156253,0.000000,1.000000


In [8]:
# Group by channel_group and user_type
grouped_df = df.groupby(['channel_group', 'user_type']).agg({
    'eur_proceeds_d100': 'sum',
    'expected_proceeds_d100': 'sum'
}).reset_index()

# Calculate total proceeds for each channel_group
total_actual_proceeds_by_channel = grouped_df.groupby('channel_group')['eur_proceeds_d100'].transform('sum')
total_expected_proceeds_by_channel = grouped_df.groupby('channel_group')['expected_proceeds_d100'].transform('sum')

# Calculate actual and expected proceeds share within each channel_group
grouped_df['actual_proceeds_share'] = (grouped_df['eur_proceeds_d100'] / total_actual_proceeds_by_channel) * 100
grouped_df['expected_proceeds_share'] = (grouped_df['expected_proceeds_d100'] / total_expected_proceeds_by_channel) * 100

# Calculate the error between expected and actual proceeds
grouped_df['proceeds_error'] = grouped_df['expected_proceeds_d100'] - grouped_df['eur_proceeds_d100']
grouped_df['error_ratio'] = grouped_df['expected_proceeds_d100'] / grouped_df['eur_proceeds_d100']
# Display the results
print("Comparison of Actual and Expected Proceeds:")
display(grouped_df)

Comparison of Actual and Expected Proceeds:


,channel_group,user_type,eur_proceeds_d100,expected_proceeds_d100,actual_proceeds_share,expected_proceeds_share,proceeds_error,error_ratio
0,affiliate_and_partnerships,day0_payer,15499.094851,15996.128081,90.720728,89.303972,497.033230,1.032069
1,affiliate_and_partnerships,other,1159.996344,1143.796212,6.789797,6.385642,-16.200132,0.986034
2,affiliate_and_partnerships,trial,425.311873,772.076315,2.489475,4.310386,346.764442,1.815318
3,brand_search,day0_payer,82907.266889,86703.784364,76.127031,66.424495,3796.517474,1.045792
4,brand_search,other,13980.640190,15661.992728,12.837290,11.998784,1681.352537,1.120263
5,brand_search,trial,12018.568526,28164.059719,11.035679,21.576722,16145.491193,2.343379
6,display_and_programmatic,day0_payer,163353.728621,165157.427301,65.787013,50.594833,1803.698680,1.011042
7,display_and_programmatic,other,38957.223757,37674.012155,15.689139,11.541173,-1283.211602,0.967061
8,display_and_programmatic,trial,45996.003213,123599.969070,18.523848,37.863994,77603.965857,2.687189
9,generic_search,day0_payer,20254.300101,21714.928728,77.280807,69.211880,1460.628627,1.072114
